<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Nama: Derfansyah Guswiranata Kurnia
* Identitas Dicoding : dgkurnia
* email dicoding : weerakurnia@gmail.com
* Alamat: Jalan Malaka Hijau No 39 Perumahan Malaka Country Estate Jakarta Timur

#Pembaruan

In [1]:
#perkembangan google colab
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,269 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,647 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [2]:
#request dan pandas
!pip install requests pandas

In [3]:
#instalasi sastarawi
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.4 MB/s eta 0:00:00


In [4]:
#instalasi wordcloud
!pip install wordcloud

In [5]:
#instalasi backoff
!pip install backoff aiohttp

# Impor Perpustakaan

In [6]:
#untuk akses ke google drive (bila diperlukan)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#instalasi perpustakaan
import requests
import json
#Bagian Pandas
import pandas as pd
pd.options.mode.chained_assignment = None #pengaturan peringatan 'rantai'
import csv
#ke manajemen

In [8]:
#manajemen data dasar
import numpy as np
seed = 0 #reprodubilitas
np.random.seed(seed)
#ke plt

In [9]:
#Bagian plt
import matplotlib.pyplot as plt
import datetime as dt #jika ada tanggal
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d
#waktu
import time
#ke sastawi, NLTK, dan wordcloud

In [10]:
#Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #bagian 'stemmer factory'
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [11]:
#sastrawi dan NITK [NITK]
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.
from nltk.corpus import stopwords  # Mengimpor modul stopwords dari pustaka NLTK.
from nltk.tokenize import word_tokenize  # Mengimpor fungsi word_tokenize dari modul tokenize dalam pustaka NLTK.

#impor wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
#ke sastrawi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Impor Teknik

In [12]:
#pemisahan data dengan 'train test split'
from sklearn.model_selection import train_test_split

In [13]:
#Teknik untuk inspeksi: perpustakaan untuk klasifikasi
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

In [14]:
#Bagian re IO bentuk string
import re
from io import StringIO

In [15]:
#metrik penilaian (akurasi, lenyap/loss, nilai f1, nilai recall)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Persiapan Ekstraksi situs

##Pendaftaran situs

In [17]:
#deklarasi 'URL target' dan asal URL
turldoma = 'https://gql.tokopedia.com/graphql/productReviewList'
asal = 'https://www.tokopedia.com' #semua dari Tokopedia
#ke pendaftaran URL

In [42]:
"""
#Daftar 'URL referensi' dan 'cookie' di Tokopedia
#daftar URL referensi
"""
urllist = ["https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-nessie-dark-green-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gruney-katun-red-m/review", "https://www.tokopedia.com/erigo/kaos-erigo-t-shirt-oversize-antelope-cotton-combed-black-m/review"]
#ke cookie

In [44]:
#pendaftaran cookie (dari atas ke bawah: a,b,c,d,e,f)
cooklist = ["_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; _hjSession_714968=eyJpZCI6ImVkMDdhZTczLTgzZmQtNDJiZS1iN2FlLTU0ZWEzMGVkMWRmMiIsImMiOjE3Mzc2NzMxOTU0OTUsInMiOjEsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxfQ==; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyvu2AJWUAQAAzRp4lRpsSK9Kq9IKEENQmF/NP0n81a4G9NW2z1k3l0EdQf8ZoypFgm3B1ftEaI4VqiM6tdlXanArsn4PVjfxjVZ37K08ajCP5hsWmqZnfgkbuPo27GCggtcgw7GC7PFKThXlJkMSawZfzEkjtIFYDoyLOXUoqnbDVarjt7IK99YGIXqrDFPs9DNyn7cqCegduQbkpKQth4SFdyMOJ17gugVxzx1GSsx+QcR1Qtfa9Mcf2mme~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737674925795.17%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737674925.57.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=231d1a0a-4caf-4293-8ee5-445a599b6786",
            "_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675011304.19%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675011.55.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=b6a6fad4-1d28-40d4-a059-e96aeb37c794",
            "_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675112541.21%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675112.19.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=db5340ce-e46c-4570-8ff2-eea15fef8204",
            "_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675229289.23%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675229.48.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=2afacd32-857a-4612-8138-0f90f5fe16cc",
            " _UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675434680.25%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675434.57.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=75225f68-83aa-47c9-8f80-0d60bfeff431",
            "_UUID_NONLOGIN_=1be9d5e19945cd39ace4288e4df11d97; _UUID_NONLOGIN_.sig=bMaPjt3Ge3Az_etBinByCA2l1rk; _SID_Tokopedia_=DtlRcaR45Ls8u2zR0EyWMaCwIb-lKujUkDrfjGHXD-LOu-8fCzNx6wcE7ORneoiESvjPzMmhkVKp_432p8WHLsvpD-z9xgx1tmq-zHGi5wycyVr7_qPcLfop7H6iXdPJ; DID=c2f41feb0dd15c191936312af3fa4e2422dd2c3bbb9f3e9541c1776d147235161cfabfb1c97d136b51b6dbc495b1e623; DID_JS=YzJmNDFmZWIwZGQxNWMxOTE5MzYzMTJhZjNmYTRlMjQyMmRkMmMzYmJiOWYzZTk1NDFjMTc3NmQxNDcyMzUxNjFjZmFiZmIxYzk3ZDEzNmI1MWI2ZGJjNDk1YjFlNjIz47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.306711698.1737673189; _gcl_au=1.1.1226888024.1737673189; ak_bmsc=F9D6DDE8942BCC4E03713784C42C8052~000000000000000000000000000000~YAAQ5i2pyvOT/5SUAQAANaZklRrhCMh7MdRKtpAkAT/TwhoJpym+giJyrAuJTOGNv+ohla9ReoK+hsgALoV7fD+GGPan21FmvmDA3Jpz1l+G8VPziqy9CfJ5rLLAcar24GlhvuIuN92yOa61cj6XP1wtwpdn2F6mYFZXvbU7QcT0+eqWjBmUDXFDkVs8D1GD5gAjSJD/VDtaRCf0prWFs3kOg16XYNTSegCScJ8Kv8L2UGkEG/B5FhTlwwIu6FzVCQW6M3ZdH3GUGgiUzL5ytrBlQyOvov2YVLwyZwAw37LCLDtbeeHV8RQTdwIHrjrzsz0Fue1Ji3/Wev90Tdl/wVsDzmZP4TISuaW9AcjIMLD3ST2Uc4pA6EulACQadieVuL3tPeZmySNEvZre0qT1ax20YjNqlWsd6eb8I0N3EphQ0QwozUoEbUNJO17X4ba/RRpGoj8tZ7JGCvF+Ga9q; _tt_enable_cookie=1; _ttp=_IQX8m81OgCxhAo11C3x8JEYJOn.tt.1; AMP_TOKEN=%24NOT_FOUND; TOPATK=ReLaxbN0Sma8Hy5pndRT5w; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImI2NWU2NDJkLWUzNjQtNWJkNS1iYmZlLTEwNmQ0NDFhMGYzMSIsImNyZWF0ZWQiOjE3Mzc2NzMxOTU0OTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=5qQ0pbbCxjoGja6Ax6qAQqJlBFCD/TpsLQvYJ5OPxJSbNnPv; _abck=9D01A68AF9735B44B661F7443776CBFC~0~YAAQCOnOF74i9I6UAQAA3fVklQ2RZncTh0GbsOGuOQhk8j87LPzIjovZ0eQzg5ndulVy7y1ZXGxKtJvvUbnmcJojUcEmmxgZLEeB07lgwLRCszxmuNAKrT/My+TBhAwTqXEqvinKZAU/jkBOPmYZZnzCMSAs1b2UM18wmrGUEXFCTtvqlZaNH5SyL5nJr39C8pYVdXQi0zs8gz5LOsxNlrKzpZ3mZiB6lOFDSgsXcZZ26pASD6Sc+FggZmDmrg7xaw5OonWfFs8I2MnhcJl9JnVHC+V5ZRAGsVhpBx8C1+aVQygF2Ran9DVqbI0szhLr++eeFSXbVLVKEnd28i0TeKOi258YEzQ8zUkXnfzXIhdh6SzfQ9SWE9mNoqM2Zza5411V7C+G55fMxN4ch4UapD+wP6RW8i/fbzajbhwZ8L90yOWFIjJNAwlSicNbi1DsLBe/6GMQPUWGSoa6DKFb5YRTVnQDws/iIwhxWtCd2ssOjXRrP/2+E2EEgaLuXvsZwZa7GDhtrhR7sOiYwGaZushVD8mVIS4BNvttUfUbOvFd8+Eumt1loi5KGmF887ayJLy2KXrgw3ZdpR/a7v0uIjdonfr2CRjFIKnanBfrBVTIPMK/K7JcqzzWXrtEpyj/NATBCkDC/kBFxe/xT7aVwzRPODrKV6pmUBmO0SFnJBz/0zKQ3gSzdmxhGQCCE68NhWOVpU0Fa72yDjNafa8=~-1~-1~-1; aus=1; _CASE_=7821624a672139213537363632303b37212f21604a67213932343b2f21674a67213931313a312f216f567367213921313331362e33322e3137573335393333393233283334393333212f216f62772139212e352d3131373433353a212f216f616f2139216e626f62686223606c6d77717a2366707762776623746a7162212f216f6c6d642139213233352d3a3733323436212f2173406c2139213230373533212f21704a67213932323630333634302f2170577a73662139216c6c60212f21744a672139332f21746b70213921585e212f21746b7070213921585e217e; _UUID_CAS_=80e2035b-0ec3-49de-9c32-8100f3d1a615; bm_sz=3CE7B2902701DC0811ED7846832A24AA~YAAQ5i2pyp6n/5SUAQAAbWB1lRozF/IxrbIMwpg86iHC/H7y3/yVRvDgxJibonZ/tuwlVJt0SOyw+VN7SOgqwBayG3Uj9/mPaQQf8YeUoWktS/DJXRUeOQXLNewrTFcqLxKCtpgWMGWx1uGRXNgbyZ11FvsVaQbNWnQ++hW4WoBOPQx15a/snrkNz7W6azb9ekdZNh0RGquEYp7PgQEe2czOQY9NppbVum8H0gV9j/78PDbyMo6JQB4QRv+TBpz16YWULhwO/QiNkJP8w1TLhAcIMT6b/K2La+RHl73HPxSubIKFEdl1bG7IvI7hK208NdsDxYgf5K2/Ll8Djxo/o9ggbJeAj0D6I3FD4ksggJIJxzYM0/1R4Qk8g9OSDKmcjaH1weAspY+vOIj+oDhtxCDqnh6BjErFYPY/gYh5tAOTTfhHtKP+NfT/PWsKdzP/d2ioEqzZfKpZdV1bLyJC0Sa/wKUmxq/rGlFN1to=~3682608~3621682; bm_sv=0087E52D3BACBBB4877168E03A2371D6~YAAQ1i2pyjjOAJWUAQAAtGaAlRpbTEBP9/HiKaOsYGziDZVk9jU6zEqoEoWY+MZYyAYSDo4mqT5sAxyJ21PWDFu8Mr1AUp+ISb/kJ2/4XVKxu2h9jxp79rj2Do4zGKQLJTOddP4KyIsjj3MsmAAgT5rJfAH0EwEBita1VzOs+Wnf72V27hw6ijN2PPIvZ+uiWklUBeoTIG84ZdY0n/Ez/JM59vEsSKHXG8N8RBWpKzcaEoZSuZZXXlZGA1FPpBC3dEnC~1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85807bf8ab475413281ab9e18cc2d094.1737673188674.1737673188674.1737675549000.28%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.88577841d29294c228b61cfba507abab.1737673195280.1737673195280.1737673208276.1%22%7D; _ga_70947XW48P=GS1.1.1737673188.1.1.1737675549.56.0.0; _ga=GA1.1.108146749.1737673189; webauthn-session=b8640a38-4a00-41d7-a451-562f2c5c349e"]

In [45]:
#persiapan 'identitas produk'
idlist =  ["7576252307", "1624334615", "7576433902",  "1624293521",  "1624323568",  "2390937942" ] #enam sampel ini dari Erigo
#urutan dari kiri ke kanan : [a,b,c,d,e,f]
#ke fragmentasi

In [46]:
#fragmentasi data untuk kemudahan
[urla, urlb, urlc, urld, urle, urlf] = urllist #daftar URL referensi
[cookiea, cookieb, cookiec, cookied, cookiee, cookief] = cooklist #daftar cookie
[ida, idb, idc, idd, ide, idf] = idlist #daftar ID produk
#ke persiapan target dan kepala

# Proses Iterasi untuk data

## Persiapan

### Pembentukan kepala dan **'bentuk kueri'**

In [49]:
#Persiapan kepala dan kueri yang distandardisasi
def stdheader(url, cookie, idpr): #idpr adalah identitas produk
  halaman_awal = 1 #di kueri
  kplstd = {'accept': '*/*',
   'accept-language': 'en-US,en;q=0.9',
   'bd-device-id': '7463084441910052353',
   'content-type': 'application/json',
   'cookie': cookie,
   'dnt': '1' ,
   'origin': asal,
   'priority': 'u=1, i',
   'referer': url,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
   'sec-ch-ua-mobile': '?0',
   'sec-ch-ua-platform': '"Windows"',
   'sec-fetch-dest': 'empty',
   'sec-fetch-mode': 'cors',
   'sec-fetch-site': 'same-site',
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
   'x-price-center': 'true',
   'x-source': 'tokopedia-lite',
   'x-tkpd-lite-service': 'zeus',
   'x-version': '1c2c0dd'}
  return kplstd

In [50]:
#variasi kepala untuk iterasi [dari indeks 0 ke lima dengan arah kiri ke kanan]
kplstdlist = [stdheader(urla, cookiea, ida), stdheader(urlb, cookieb, idb), stdheader(urlc, cookiec, idc), stdheader(urld, cookied, idd), stdheader(urle, cookiee, ide), stdheader(urlf, cookief, idf)]
#ke persiapan bentuk

### Iterasi utama

In [51]:
#Proses iterasi produk
def datagenerator (url, cookie, idpr):
  #pemangilan fungsi pertama (stdheader)
  header = stdheader(url, cookie, idpr)
  #persiapan bentuk untuk iterasi
  daftar_teks = []
  daftar_nilai = []
  daftar_label = []

  #Batas halaman untuk ulasan
  awal = 1 #dari halaman 1
  akhir = 50 #batas ulasan paling tinggi
  #iterasi
  while awal <= akhir:
    #pemformatan kueri
    fquery = f'[{{"operationName":"productReviewList","variables":{{"productID":"{idpr}","page":{awal},"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n      }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n      }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n      }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n      }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n      }}\\n      badRatingReasonFmt\\n      __typename\\n    }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n    }}\\n    hasNext\\n    totalReviews\\n    __typename\\n  }}\\n}}\\n"}}]'
    query = fquery #dari masing-masing hasil
    response = requests.post(turldoma, headers=header, data=query)
    daftar_ulasan = response.json()[0]['data']['productrevGetProductReviewList']['list']
    for review in daftar_ulasan:
        daftar_teks.append(review['message'])
        daftar_nilai.append(review['productRating'])
        if review['productRating'] == 0:
            daftar_label.append('total disappointment') #hal yang mengecewakan sekali
        elif review['productRating'] == 1:
            daftar_label.append('so inferior') #hal sangat inferior
        elif review['productRating'] == 2:
            daftar_label.append('very low') #sangat rendah
        elif review['productRating'] == 3:
            daftar_label.append('average') #rata-rata
        elif review['productRating'] == 4:
            daftar_label.append('good') #sangat bagus
        elif review['productRating'] == 5:
            daftar_label.append('excelent') #sempurna
        else:
            daftar_label.append('No score') #tak mau diulas
        awal += 1
  return daftar_teks, daftar_nilai, daftar_label
#ke eksekusi untuk penggabungan

In [ ]:
#Proses pembentukan daftar dari enam produk untuk disatukan (Bagian 1)
teks1, nilai1, label1 = datagenerator(urla, cookiea, ida) #ProdukA
teks2, nilai2, label2 = datagenerator(urlb, cookieb, idb) #ProdukB
teks3, nilai3, label3 = datagenerator(urlc, cookiec, idc) #ProdukC
#ke bagian 2

In [ ]:
#Bagian 2
teks4, nilai4, label4 = datagenerator(urld, cookied, idd) #ProdukD
#ke bagian 3

In [ ]:
#Bagian 3
teks5, nilai5, label5 = datagenerator(urle, cookiee, ide) #ProdukE
teks6, nilai6, label6 = datagenerator(urlf,cookief,idf)

### Penyatuan ke satu data

In [ ]:
#inspeksi bentuk data


In [ ]:
#deklarasi total ulasan
sampteks = teks1 + teks2 + teks3 + teks4 + teks5 + teks6 #bagian teks
sampnilai = nilai1 + nilai2 + nilai3 + nilai4 + nilai5 + nilai6 #bagian nilai
samplabel = label1 + label2 + label3 + label4 + label5 + label6 #bagian label
#ke penyatuan

enam ulasan ini berasal dari enam produk berasal dari Erigo

In [ ]:
#penyatuan ke dataframe
dfori = pd.DataFrame({'Pesan': sampteks,'Nilai': sampnilai,'Label': samplabel})
dfkopi = dfori.copy()
#cek panjang
lendfkop = len(dfkopi)
print(lendfkop)

300


#### Inspeksi hasil data dari kesalahan

In [ ]:
#cek sampel data
kepdfkopi = dfkopi.head(7)
kepdfkopi

,Pesan,Nilai,Label
0,barang bagus kualitas nya top harga nya naik o...,5,excelent
1,"modelnya keren, bahannya bagus cuma agak kepan...",5,excelent
2,"tinggi 175 bb 60 ukuran L cocok si, pas dan en...",5,excelent
3,enak di pake adem dan kece pas di badan gw. ta...,5,excelent
4,APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa r...,5,excelent
5,"oke paket di terima dengan baik ,, untuk kuali...",5,excelent
6,"pas, bahannya enak banget dibadan dan juga ker...",5,excelent


In [ ]:
#cek kolom
dfkopinfo = dfkopi.info()
print(dfkopinfo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Pesan   300 non-null    object
 1   Nilai   300 non-null    int64 
 2   Label   300 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.2+ KB
None


#### Pembersihan data (bebas dari data duplikat dan kosong)

In [ ]:
#Data Kosong
totkos = dfkopi.isnull().sum() #Kosong

In [ ]:
#inspeksi data hilang
tothil = dfkopi.isna().sum() #Hilang
print(tothil)

Pesan    0
Nilai    0
Label    0
dtype: int64


In [ ]:
#inspeksi data duplikat
dfkopi.duplicated().sum()

260

In [ ]:
#Pembersihan data duplikat
dfkopi.drop_duplicates(inplace=True)
dfkopitung = dfkopi.copy()
kep2 = dfkopitung.head(7) #pasca pembersihan
print(kep2)

                                               Pesan  Nilai     Label
0  barang bagus kualitas nya top harga nya naik o...      5  excelent
1  modelnya keren, bahannya bagus cuma agak kepan...      5  excelent
2  tinggi 175 bb 60 ukuran L cocok si, pas dan en...      5  excelent
3  enak di pake adem dan kece pas di badan gw. ta...      5  excelent
4  APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa r...      5  excelent
5  oke paket di terima dengan baik ,, untuk kuali...      5  excelent
6  pas, bahannya enak banget dibadan dan juga ker...      5  excelent


In [ ]:
#cek panjang
lendfkop2 = len(dfkopitung)
print(lendfkop2)

40


# Persiapan tahapan ulasan

## Penataan ulang kata-kata

In [ ]:
#pembuatan daftar kata-kata bahasa gaul
kata_gaul = ['yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'untuk', 'dengan', 'saya', 'ada', 'akan', 'pada',
             'seperti', 'karena', 'atau', 'jika', 'tidak', 'dalam','iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"] #daftar kata gaul
#Bahasa gaul yang populer